In [1]:
import re

import xlwings as xw
import pandas as pd
from shapely.wkt import loads
from win32com.universal import com_error

from src.parsing import Splitter, Parser, Templates

In [24]:
sheet_taxation_list = xw.sheets['Ведомость']
taxation_list_df = sheet_taxation_list.range('A1').expand().options(pd.DataFrame, header=1).value
taxation_list_df = taxation_list_df[
    ['Номер точки', 'Наименование', 'Количество', 'Высота', 'Толщина', 'Состояние', 'Кустарник']]

taxation_list_df

,Номер точки,Наименование,Количество,Высота,Толщина,Состояние,Кустарник
Индекс,,,,,,,
1.0,1,береза,14 м2,1,0.1,Хорошее,0.0
2.0,2,ольха,14 м2,2.5,0.08,Хорошее,0.0
3.0,"3,4",Ольха серая,2,"5,5","4,5",Хорошее,0.0
4.0,5,береза,6 стволов,"4,4,4,4,4,4","5,5,5,5,5,5",Хорошее,0.0
5.0,6,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7х3,4х11, 2х40",Хорошее,0.0
6.0,7,береза,1,2.5,0.05,Хорошее,0.0
7.0,8,береза,72 м2,1.5,0.05,Хорошее,0.0
8.0,9,ольха,72 м2,2,0.06,Хорошее,0.0
9.0,10,Яблоня домашняя,2 ствола,1.5х2,0.04х2,Хорошее,0.0


In [31]:
def get_shapes_from_autocad_df(df: pd.DataFrame, number: str) -> dict:
    """
    Получение геометрии из датафрейма "Автокад" по номеру точки
    Args:
        df (pd.DataFrame): Датафрейм "Автокад"
        number (str): Номер точки

    Returns:
        dict: Словарь двух колонок для датафрейма со списками позиций номеров и геометрий
    """
    number_positions, geometries = [], []
    split_numbers = Splitter.number(number)
    df = df.set_index('Разделенный номер')
    for split_number in split_numbers:
        shapes = df.loc[split_number][['Позиция номера', 'Геометрия']].to_dict()
        number_positions.append(shapes['Позиция номера'])
        geometries.append(shapes['Геометрия'])
    return {'Список позиций номеров': number_positions, 'Список геометрии': geometries}

def split_taxation_list_item(df: pd.DataFrame, series: pd.Series) -> list[dict]:
    """
    Получение строки ОРМ из строки таблицы Ведомости

    Args:
        df (pd.DataFrame): Датафрейм листа Автокад
        series (pd.Series): Строка таблицы

    Returns:
        list[dict]: Список словарей ОРМ для датафрейма
    """
    match_trunk = re.search(Templates.TRUNKS, series['Количество'])
    match_contour = re.search(Templates.CONTOUR, series['Количество'])
    match_line = re.search(Templates.LINE, series['Количество'])
    shapes = get_shapes_from_autocad_df(df, series['Номер точки'])
    numbers_positions, geometries = shapes['Список позиций номеров'], shapes['Список геометрии']
    if not match_contour and not match_line and not match_trunk:
        split_numbers = Splitter.number(series['Номер точки'])
        split_height = Splitter.size(series['Высота'])
        split_diameter = Splitter.size(series['Толщина'])
        split_quality = Splitter.quality(series['Состояние'])
        is_stump = Parser.identification_stump(series['Высота'], series['Толщина'], bool(series['Кустарник']))
        if len(split_numbers) == 1:
            if (not is_stump or "пень" in series['Наименование'].lower()) and len(split_quality) == 1 and len(
                    numbers_positions) == 1 and len(geometries) == 1:
                series_dict = series.to_dict()
                series_dict['Позиция номера'] = numbers_positions[0]
                series_dict['Геометрия'] = geometries[0]
                return [series_dict]
        else:
            if (not is_stump or "пень" in series['Наименование'].lower()) and len(split_quality) == 1 and len(
                    numbers_positions) == 1 and len(geometries) == 1:
                series_dict = series.to_dict()
                series_dict['Позиция номера'] = numbers_positions[0]
                series_dict['Геометрия'] = geometries[0]
                return [series_dict]
            if len(split_height) == 1:
                split_height = split_height * int(series['Количество'])
            if len(split_diameter) == 1:
                split_diameter = split_diameter * int(series['Количество'])
            if len(split_quality) == 1:
                split_quality = split_quality * int(series['Количество'])
            series_data = []
            for idx in range(len(split_numbers)):
                if "пень" not in series['Наименование'].lower():
                    is_stump = Parser.identification_stump(split_height[idx], split_diameter[idx],
                                                           bool(series['Кустарник']))
                    name = series['Наименование'] + " (пень)" if is_stump else series['Наименование']
                else:
                    name = series['Наименование']
                series_data.append({
                    'Номер точки': split_numbers[idx],
                    'Наименование': name,
                    'Количество': 1,
                    'Высота': split_height[idx],
                    'Толщина': split_diameter[idx],
                    'Состояние': split_quality[idx],
                    'Кустарник': series['Кустарник'],
                    'Позиция номера': numbers_positions[idx],
                    'Геометрия': geometries[idx]
                })
            return series_data
    else:
        series_dict = series.to_dict()
        series_dict['Позиция номера'] = numbers_positions[0]
        series_dict['Геометрия'] = geometries[0]
        return [series_dict]

In [29]:
sheet_autocad = xw.sheets['Автокад']
autocad_df = sheet_autocad.range('A1').expand().options(pd.DataFrame, header=1, index=False).value
autocad_df['Позиция номера'] = autocad_df['Позиция номера'].apply(lambda x: loads(x))
autocad_df['Геометрия'] = autocad_df['Геометрия'].apply(lambda x: loads(x))

assert autocad_df['Разделенный номер'].is_unique

autocad_df

,Индекс,Номер точки,Разделенный номер,Позиция номера,Геометрия,Размер
0,0.0,3,3,POINT (3108.838205298518 1707.285109871409),POINT (3108.838205298518 1707.285109871409),NaN
1,1.0,4,4,POINT (3114.150988844972 1707.4695572873902),POINT (3114.150988844972 1707.4695572873902),NaN
2,2.0,5,5,POINT (3115.690642463274 1707.5867820573599),POINT (3115.690642463274 1707.5867820573599),NaN
3,3.0,6,6,POINT (3115.1004453425708 1706.18062074892),POINT (3115.1004453425708 1706.18062074892),NaN
4,4.0,7,7,POINT (3114.699609216242 1705.0905155704238),POINT (3114.699609216242 1705.0905155704238),NaN
5,5.0,10,10,POINT (3104.043722283615 1698.543651078497),POINT (3104.043722283615 1698.543651078497),NaN
6,6.0,11,11,POINT (3105.090899215862 1695.9481208535972),POINT (3105.090899215862 1695.9481208535972),NaN
7,7.0,12,12,POINT (3106.332887793542 1693.7013226121771),POINT (3106.332887793542 1693.7013226121771),NaN
8,8.0,14,14,POINT (3104.361115387819 1692.888423763281),POINT (3104.361115387819 1692.888423763281),NaN
9,9.0,13,13,POINT (3107.4888707400646 1689.420932022136),POINT (3107.4888707400646 1689.420932022136),NaN


In [32]:
taxation_list_orm = []
for _, series in taxation_list_df.iterrows():
    taxation_list_orm.extend(split_taxation_list_item(autocad_df, series))

taxation_list_orm_df = pd.DataFrame(taxation_list_orm)

taxation_list_orm_df

,Номер точки,Наименование,Количество,Высота,Толщина,Состояние,Кустарник,Позиция номера,Геометрия
0,1,береза,14 м2,1,0.1,Хорошее,0.0,POINT (3075.965582309878 1696.6051757815592),POLYGON ((3075.045526933311 1693.2592293092591...
1,2,ольха,14 м2,2.5,0.08,Хорошее,0.0,POINT (3074.823165523758 1693.9006874227402),POLYGON ((3075.045526933311 1693.2592293092591...
2,3,Ольха серая,1,5,4,Хорошее,0.0,POINT (3108.838205298518 1707.285109871409),POINT (3108.838205298518 1707.285109871409)
3,4,Ольха серая,1,5,5,Хорошее,0.0,POINT (3114.150988844972 1707.4695572873902),POINT (3114.150988844972 1707.4695572873902)
4,5,береза,6 стволов,"4,4,4,4,4,4","5,5,5,5,5,5",Хорошее,0.0,POINT (3115.690642463274 1707.5867820573599),POINT (3115.690642463274 1707.5867820573599)
5,6,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7х3,4х11, 2х40",Хорошее,0.0,POINT (3115.1004453425708 1706.18062074892),POINT (3115.1004453425708 1706.18062074892)
6,7,береза,1,2.5,0.05,Хорошее,0.0,POINT (3114.699609216242 1705.0905155704238),POINT (3114.699609216242 1705.0905155704238)
7,8,береза,72 м2,1.5,0.05,Хорошее,0.0,POINT (3106.665839634858 1703.063361392738),"POLYGON ((3106.916523995033 1690.249262830753,..."
8,9,ольха,72 м2,2,0.06,Хорошее,0.0,POINT (3104.129760581689 1690.6193231862521),"POLYGON ((3106.916523995033 1690.249262830753,..."
9,10,Яблоня домашняя,2 ствола,1.5х2,0.04х2,Хорошее,0.0,POINT (3104.043722283615 1698.543651078497),POINT (3104.043722283615 1698.543651078497)
